# Setting

In [1]:
import pymc3 as pm
import arviz
from matplotlib import gridspec
from matplotlib.lines import Line2D
from scipy import stats

import numpy as np
import pandas as pd

In [2]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [3]:
def print_interval(low, high):
    #low, high = low*100, high*100
    p = (low+high)/2
    err = high - p
    #print(f'confidence interval: {p:.0f} ± {err:.2f} % [{low:.0f}, {high:.0f}]')
    print(f'confidence interval: {p:.3f} ± {err:.4f}, [{low:.3f}, {high:.3f}]')
    
    
def test_hypothesis(pvalue, level=0.95):
    alpha = 1 - c_level
    if pvalue > alpha:
        s = 'Accept'
    else:
        s = 'Reject'
    print(f'{s} H0 (p-value = {pvalue:.4e})')

# Item 1

18~49세 전국 성인 남녀 1000명이 조사에 응해주었다. 응답률은 5.4%, 표본오차는 95% 신뢰수준에서 ±3.1%포인트다
미혼 응답자 579명 중 결혼 의향이 있다는 사람은 56.5%, 의향이 없다는 사람은 43.5%였다. 
그런데 여기서 남녀 성별 격차가 나타난다. 미혼 남성은 65.7%가 결혼하고 싶다고 답한다. 
그러나 여성은 54.5%가 결혼할 의향이 없다고 답한다.
(미혼 여성은 45.5%가 결혼하고 싶다고 답한다)
이 성별 격차는 결혼 적령기인 30대에서 가장 크다. 30대 미혼 남성은 71.1%가 결혼 의향이 있지만, 30대 미혼 여성 55.3%는 결혼 의향이 없다.
20대 역시 남성 64.6%가 결혼할 의향이 있지만, 여성은 52.7%가 의향이 없다고 답했다.

In [4]:
n_all = 1000
n_single = 579 # 미혼남녀
c_level = .95 # 신뢰수준 95%
alpha = 1 - c_level 

In [5]:
# 20대 여성 응답자(160명)
n_20s_f = 160

In [6]:
# 나이당 인원 가정: 18~49세 전국 성인 남녀 1000명이 조사에 응해주었다
n_age = n_all / (49-18+1) # 나이당 남녀수
n_age_single = n_single / (49-18+1) # 나이당 싱글남녀수

## ST01
성인 남녀 1000명이 조사에 응해주었다. 응답률은 5.4%, 표본오차는 95% 신뢰수준에서 ±3.1%포인트다

Result:
- 한 yes/no 문항에 대하여 1000명중 절반이 긍정한 경우 표본오차가 최대가 되며 신뢰수준 95%에서는 표본오차 ±3.1%

In [5]:
k = 500 # 어떤 한 질문에 절반이 yes로 대답하는 경우
n = n_all

Confidence interval for a binomial proportion with asymtotic normal approximation

In [6]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
low, high

(0.4690102483847719, 0.5309897516152281)

In [7]:
p = k/n
p-low, high-p

(0.0309897516152281, 0.0309897516152281)

In [8]:
# 표본오차 ±3.1%포인트는 긍정/부정 설문에 50%가 긍정했을때의 오차인 셈
print_interval(low, high)

confidence interval: 0.500 ± 0.0310, [0.469, 0.531]


In [9]:
# 70%가 긍정시 표본오차는 2.8%
p = 0.7
low, high = proportion_confint(int(p*n), n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.700 ± 0.0284, [0.672, 0.728]


## ST02
미혼 응답자 579명 중 결혼 의향이 있다는 사람은 56.5%

Result:
- 가설검정 결과 미혼의 결혼의향 비율은 50%가 아니다.
- 가설검정은 두가지 방법으로 구현하였는데 p-value는 상이하지만 모두 유의수준보다 충분히 작다.
- '50%가 아니다'와 '50%보다 크다'의 두가지 대립가설에 대하여 전자(양측검정)는 p값은 크지만(물론 유의수준 미만) 표본오차가 작고, 후자(단측검정)는 p값은 작지만 오차가 크다. 단측검정은 크다(작다)는 확신이 있을때 최소값(최대값)을 확인할 때 사용.
- 표본오차는 두가지 방법으로 구현했는데 오차범위 유사.
- 표본오차 계산시 proportion_confint는 양측검정에만 사용가능(asymptotic normal approximation)
- 미혼 결혼의향의 오차는 ±4.0%로 전체 1000명에 대한 문항의 최대오차(±3.1%) 보다 크다.
- 미혼 결혼의향 비율은 최소 52.4%, 최대 60.5% (95% 신뢰수준, 양측검정)
- 단측 검정시 미혼 결혼의향의 최소비율은 53% (95% 신뢰수준)

In [28]:
n = n_single
p_obs = .565
k = int(n*p_obs) # 결혼의향 있는 사람 수

### CS1 
Confidence interval for a binomial proportion with asymptotic normal approximation

In [29]:
method = 'normal'
#method = 'binom_test' # killed due to small mem size

In [30]:
# 결혼의향 비율에 대한 오차는 ±4%로 설문조사 처음 언급한 표본오차 ±3.1%보다 큼
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.565 ± 0.0404, [0.524, 0.605]


### CS2
binomial test
- H0: 결혼의향 비율은 50%
- Ha: 결혼의향 비율은 50%가 아니다

In [31]:
p = .5
alternative = 'two-sided'

In [32]:
result = stats.binomtest(k, n, p, alternative=alternative)
result

BinomTestResult(k=327, n=579, alternative='two-sided', proportion_estimate=0.5647668393782384, pvalue=0.0020762662419686645)

In [33]:
# reject H0: 결혼의향 p 가정시 p_obj(k/n)일 가능성은 5% 보다 매우 작다
test_hypothesis(result.pvalue)

Reject H0 (p-value = 2.0763e-03)


In [34]:
# proportion_confint과 유사한 결과 (see CS1)
ci = result.proportion_ci(confidence_level=c_level)
print_interval(ci.low, ci.high)

confidence interval: 0.564 ± 0.0412, [0.523, 0.606]


### CS3
- H0: 결혼의향 비율은 50%
- Ha: 결혼의향 비율은 50% 초과

In [35]:
p = .5
alternative = 'greater'

In [36]:
result = stats.binomtest(k, n, p, alternative=alternative)
result

BinomTestResult(k=327, n=579, alternative='greater', proportion_estimate=0.5647668393782384, pvalue=0.0010381331209843323)

In [37]:
# reject H0
test_hypothesis(result.pvalue)

Reject H0 (p-value = 1.0381e-03)


In [38]:
# CS2에 비해 p값은 더 작으나(더 확실하게 H0를 기각) 신뢰구간은 더 넓어짐
# see https://stats.oarc.ucla.edu/other/mult-pkg/faq/general/faq-what-are-the-differences-between-one-tailed-and-two-tailed-tests/
ci = result.proportion_ci(confidence_level=c_level)
print_interval(ci.low, ci.high)

confidence interval: 0.765 ± 0.2351, [0.530, 1.000]


### CS4
Test for proportions based on normal (z) test
- H0: 결혼의향 비율은 50%
- Ha: 결혼의향 비율은 50%가 아니다

In [24]:
p = .5
alternative='two-sided'
#alternative='larger'

In [25]:
# reject H0
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 1.6701e-03)


## ST03
미혼 남성은 65.7%가 결혼하고 싶다고 답한다.

Result:
- 데이터가 없어 미혼인의 남녀비율을 동일하게 가정.
- 가설검정 결과 미혼남성의 결혼의향 비율은 미혼전체 결혼의향 비율과 같지 않다. 
- 미혼남성 결혼의향의 오차는 ±5.5%, 범위는 59~71% (95% 신뢰수준, 양측검정)

In [50]:
n = int(n_single/2) # 미혼남녀 동일비율로 가정
p_obs = .657
k = int(n*p_obs)

### CS1 (standard)
- H0: 결혼의향 비율은 56.5% (남녀전체 비율)
- Ha: 결혼의향 비율은 56.5%이 아니다

In [51]:
p = .565
alternative = 'two-sided'

In [52]:
# reject H0
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 1.4736e-03)


In [53]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.654 ± 0.0548, [0.599, 0.709]


additional

In [54]:
result = stats.binomtest(k, n, p, alternative=alternative)
ci = result.proportion_ci(confidence_level=c_level)
print_interval(ci.low, ci.high)

confidence interval: 0.652 ± 0.0563, [0.596, 0.709]


### CS2
- H0: 결혼의향 비율은 50%
- Ha: 결혼의향 비율은 50%가 아니다

In [32]:
p = .5
alternative = 'two-sided'

In [33]:
# reject H0
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 3.7396e-08)


## ST04
여성은 54.5%가 결혼할 의향이 없다고 답한다.
(미혼 여성은 45.5%가 결혼하고 싶다고 답한다)

Result:
- 데이터가 없어 미혼인의 남녀비율을 동일하게 가정.
- 가설검정 결과 미혼여성의 결혼의향 비율은 미혼전체 결혼의향 비율과 같지 않다. 
- 미혼여성 결혼의향의 오차는 ±5.7%, 범위는 40~51% (95% 신뢰수준, 양측검정)
- 미혼중 여성비율을 30%로 가정시 결혼의향의 오차는 7.4%, 범위는 38~53% (95% 신뢰수준, 양측검정)

In [5]:
n = int(n_single/2) # 미혼남녀 동일비율로 가정
p_obs = .455
k = int(n*p_obs)

### CS1
- H0: 결혼의향 비율은 56.5% (남녀전체 비율)
- Ha: 결혼의향 비율은 56.5%이 아니다

In [35]:
p = .565
alternative = 'two-sided'

In [36]:
# reject H0
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 1.3623e-04)


In [37]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.453 ± 0.0574, [0.396, 0.511]


### CS2
결혼의향 없음

In [55]:
p_obs = .545
k = int(n*p_obs)

In [56]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.543 ± 0.0574, [0.486, 0.601]


### CS3
결혼의향 있고, 여성비율 새로 가정

In [39]:
rf = 0.3 # 미혼중 여성비율 가정
n = int(n_single*rf)
p_obs = .455
k = int(n*p_obs)

In [8]:
p = .565
alternative = 'two-sided'

In [9]:
# reject H0
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 2.5531e-03)


In [10]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.451 ± 0.0741, [0.377, 0.525]


## ST05
30대 미혼 남성은 71.1%가 결혼 의향이 있지만, 30대 미혼 여성 55.3%는 결혼 의향이 없다

Result:
- 데이터가 없어 미혼인의 남녀비율을 동일하게 가정.
- 가설검정 결과 30대 미혼남성의 결혼의향 비율은 전체 미혼남성의 결혼의향 비율(65.7%)과 같다. 
- 30대 미혼남성 결혼의향의 오차는 ±9.5%, 범위는 61~80% (95% 신뢰수준, 양측검정)
- 가설검정 결과 30대 미혼여성의 결혼의향 비율은 전체 미혼여성의 결혼의향 비율(45.5%)과 같다. 
- 30대 미혼여성 결혼의향의 오차는 ±10%, 범위는 44~65% (95% 신뢰수준, 양측검정)

### CS1
- H0: 30대 미혼남성과 전체 미혼남성의 결혼의향 비율은 동일하다.
- Ha: 결혼의향 비율은 다르다

In [10]:
n = int(n_age_single*10/2) # 30대 미혼남 (남녀 동일비율 가정)
p_obs = .711
k = int(n*p_obs)

In [7]:
p = .657 # 미혼 남성은 65.7%가 결혼하고 싶다고 답한다.
alternative = 'two-sided'

In [8]:
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Accept H0 (p-value = 3.7337e-01)


In [9]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.700 ± 0.0947, [0.605, 0.795]


### CS2
- H0: 30대 미혼여성과 전체 미혼여성의 결혼의향 비율은 동일하다.
- Ha: 결혼의향 비율은 다르다

In [11]:
n = int(n_age_single*10/2) # 30대 미혼녀 (남녀 동일비율 가정)
p_obs = .553
k = int(n*p_obs)

In [12]:
p = .455 # 미혼 여성은 45.5%가 결혼하고 싶다고 답한다.
alternative = 'two-sided'

In [13]:
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Accept H0 (p-value = 8.8412e-02)


In [14]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.544 ± 0.1029, [0.442, 0.647]


## ST06
20대 역시 남성 64.6%가 결혼할 의향이 있지만, 여성은 52.7%가 의향이 없다고 답했다.

Result:
- 20대 여성 중 미혼의 비율은 전체 미혼 비율과 동일하다고 가정
- 가설검정 결과 20대 미혼여성의 결혼의향 비율(47.3%)은 전체 미혼여성의 결혼의향 비율(45.5%)과 같다. 
- 20대 미혼여성 결혼의향의 오차는 ±10%, 범위는 37~57% (95% 신뢰수준, 양측검정)
- 20대 여성 모두 미혼으로 가정해도 유사한 결과를 보임

### CS1
- H0: 20대 미혼여성과 전체 미혼여성의 결혼의향 비율은 동일하다.
- Ha: 결혼의향 비율은 다르다

In [7]:
n = int(n_20s_f*(n_single/n_all)) # 20대 미혼녀 (20대 여자의 미혼 비율을 전체 미혼비율과 동일하다고 가정)
p_obs = 1 - .527
k = int(n*p_obs)

In [8]:
p = .455 # 미혼 여성은 45.5%가 결혼하고 싶다고 답한다.
alternative = 'two-sided'

In [9]:
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Accept H0 (p-value = 8.1172e-01)


In [10]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.467 ± 0.1020, [0.365, 0.569]


### CS2
20대 여자의 모두가 미혼이라고 가정
- H0: 20대 미혼여성과 전체 미혼여성의 결혼의향 비율은 동일하다.
- Ha: 결혼의향 비율은 다르다

In [11]:
n = n_20s_f # 20대 여자 모두 미혼으로 가정
p_obs = 1 - .527
k = int(n*p_obs)

In [12]:
p = .455 # 미혼 여성은 45.5%가 결혼하고 싶다고 답한다.
alternative = 'two-sided'

In [ ]:
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Accept H0 (p-value = 7.2744e-01)


In [ ]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.469 ± 0.0773, [0.391, 0.546]


# Item 2

- (그림5) 20대 여성의 49.2%, 30대 여성의 21.4%가 페미니즘에 긍정적 모습을 보인다. 반면 남성의 경우, 20대 9.5%, 30대 6.2%만 페미니즘에 긍정적 태도를 가진다.
- 페미니즘은 결혼 의향과 상당한 상관관계를 보인다. 페미니즘에 긍정적인 응답자 가운데 70.1%가 결혼 의향이 없다고 밝혔다. 반면 페미니즘에 ‘보통’이거나 ‘부정적’인 태도를 갖는 사람들은 각각 63.5%, 65.2%가 결혼 의향이 있다고 응답했다. => 그림5로 확인불가
- (그림5) 전체 응답자중 18.8%가 페미니즘에 긍정적, 41.3%가 부정적
- (그림5) 결혼의향 없는 미혼 중 37%가 페미니즘에 긍정적, 34.3%가 부정적

In [16]:
n_all = 1000
n_single = 579 # 미혼남녀
c_level = .95 # 신뢰수준 95%
alpha = 1 - c_level 

In [17]:
# 20대 여성 응답자(160명)
n_20s_f = 160

In [18]:
# 나이당 인원 가정: 18~49세 전국 성인 남녀 1000명이 조사에 응해주었다
n_age = n_all / (49-18+1) # 나이당 남녀수
n_age_single = n_single / (49-18+1) # 나이당 싱글남녀수

In [19]:
# 미혼 응답자 579명 중 결혼 의향이 있다는 사람은 56.5%, 의향이 없다는 사람은 43.5%였다. 
n_single_marry = round(n_single * .567)
n_single_nomarry = n_single - n_single_marry

In [28]:
n_fem = round(n_all * 0.188) # 전체중 페미니즘에 긍정적 인원수
n_single_nomarry_fem = round(n_single_nomarry * .37) # 결혼의향 없는 미혼중 페미니즘에 긍정적
n_single_nomarry_fema = round(n_single_nomarry * .343) # 결혼의향 없는 미혼중 페미니즘에 부정적

In [21]:
n_fem, n_single_nomarry_fem

(188, 93)

## ST01
20대 여성의 49.2%, 30대 여성의 21.4%가 페미니즘에 긍정적 모습을 보인다. 반면 남성의 경우, 20대 9.5%, 30대 6.2%만 페미니즘에 긍정적 태도를 가진다.

Result:
- 가설검정 결과 20대 여성은 전체 인원보다 페미니즘에 더 긍정적, 20대 남성은 덜 긍적적

### CS1
- H0: 20대 여성의 긍정 비율은 전체 긍정 비율과 동일하다.
- Ha: 서로 다르다

In [22]:
n = n_20s_f # 20대 여성수
p_obs = .492
k = int(n*p_obs) # 페미니즘에 긍정적 인원수

In [23]:
p = .188 # 전체 긍적적 비율.
alternative = 'two-sided'

In [24]:
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 3.4773e-14)


In [25]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.487 ± 0.0775, [0.410, 0.565]


### CS2
- H0: 20대 남성의 긍적 비율은 전체 긍정 비율과 동일하다.
- Ha: 서로 다르다

In [25]:
n = int(n_age * 10 / 2) # 20대 남성수 가정
p_obs = .095
k = int(n*p_obs) # 페미니즘에 긍정적 인원수

In [26]:
p = .188 # 전체 긍적적 비율.
alternative = 'two-sided'

In [27]:
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 1.7566e-05)


In [28]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.090 ± 0.0449, [0.045, 0.135]


## ST02
결혼의향 없는 미혼 중 37%가 페미니즘에 긍정적, 34.3%가 부정적

Result:
- 비혼주의자 미혼의 페미니즘 긍정적 비율은 전체 인원의 긍정적 비율보다 높다
- 비혼주의자 미혼의 페미니즘 부정적 비율은 전체 인원의 부정적 비율보다 낮다
- 전반적으로 페미니즘에 대한 인식이 비혼결정에 영향을 주는것으로 보인다. 베이지안(조건부확률)을 적용할수있을까?

### CS1
- H0: 결혼의향 없는 미혼의 긍정 비율은 전체 긍정 비율과 동일하다.
- Ha: 서로 다르다

In [34]:
n = n_single_nomarry # 결혼의향없는 미혼
p_obs = .37
k = int(n*p_obs) # 페미니즘에 긍정적 인원수

In [35]:
p = .188 # 전체 긍적적 비율.
alternative = 'two-sided'

In [36]:
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 4.3577e-09)


In [37]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.367 ± 0.0596, [0.307, 0.426]


### CS2
- H0: 결혼의향 없는 미혼의 부정적 비율은 전체 부정적 비율과 동일하다.
- Ha: 서로 다르다

In [38]:
n = n_single_nomarry # 결혼의향없는 미혼
p_obs = .343
k = int(n*p_obs) # 페미니즘에 부정적 인원수

In [39]:
p = .413 # 전체 부정적 비율.
alternative = 'two-sided'

In [40]:
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 1.8816e-02)


In [41]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.343 ± 0.0587, [0.284, 0.401]


## ST03
페미니즘은 결혼 의향과 상당한 상관관계를 보인다. 페미니즘에 긍정적인 응답자 가운데 70.1%가 결혼 의향이 없다고 밝혔다. 반면 페미니즘에 ‘보통’이거나 ‘부정적’인 태도를 갖는 사람들은 각각 63.5%, 65.2%가 결혼 의향이 있다고 응답했다.

Result:
- 결혼의향 질문이므로 대상은 미혼을 가정
- 가설검정 결과 결혼의향 없는 미혼은 페미니즘에 대한 긍정/부정 비율은 동일하다. 즉 결혼을 안하는 이유는 페미니즘이라고 할 수 없다
- 가설검정 결과 결혼의향 있는 미혼은 페미니즘에 대한 부정 비율이 높다. (페미니즘이 싫어서 결혼하는 것은 아닐테고 뭘 알수 있지?)
- 카이제곱 검정 결과 결혼의향에 따라 페미니즘에 대한 인식의 차이가 있다.
- 카이제곱 검정 결과 결혼의향 있을때 페미니즘에 대한 인식의 차이가 있다.
- 카이제곱 검정 결과 결혼의향 없을때 페미니즘에 대한 인식의 차이는 없다.

### CS1
- H0: 결혼의향 없는 미혼중 페미니즘에 긍정적인 비율은 부정적인 비율과 동일하다.
- Ha: 서로 다르다

In [50]:
n = n_single_nomarry # 결혼의향없는 미혼
k = n_single_nomarry_fem # 그중 페미니즘에 긍정적 인원수
n,k

(251, 93)

In [40]:
p = n_single_nomarry_fema/n # 결혼의향없는 미혼의 페미니즘 긍정 비율.
alternative = 'two-sided'

In [41]:
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Accept H0 (p-value = 3.6025e-01)


In [42]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.371 ± 0.0597, [0.311, 0.430]


additional

In [43]:
k = n_single_nomarry_fema # 결혼의향없는 미혼의 페미니즘 긍정 인원수
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.343 ± 0.0587, [0.284, 0.401]


### CS2
- H0: 결혼의향 있는 미혼중 페미니즘에 긍정적인 비율은 부정적인 비율과 동일하다.
- Ha: 서로 다르다

In [53]:
n = n_single_marry # 결혼의향있는 미혼
p = 0.701 # 페미니즘에 긍정적인 응답자 가운데 70.1%가 결혼 의향이 없다고 밝혔다
k = round(n_single_nomarry_fem * (1/p-1)) # 결혼의향있는 미혼중 페미니즘 긍정 인원
k, n

(40, 328)

In [60]:
# 페미니즘에 ‘보통’이거나 ‘부정적’인 태도를 갖는 사람들은 각각 63.5%, 65.2%가 결혼 의향이 있다
p = .652
n_single_marry_fema = round(n_single_nomarry_fema * p / (1-p)) # 결혼의향있는 미혼의 페미니즘 부정 인원
p = n_single_marry_fema/n # 그 비율
alternative = 'two-sided'
p

0.49085365853658536

In [61]:
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 1.1763e-92)


In [62]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.122 ± 0.0354, [0.087, 0.157]


additional

In [63]:
k = n_single_marry_fema # 결혼의향있는 미혼의 페미니즘 부정 인원수
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.491 ± 0.0541, [0.437, 0.545]


### CS3
Chi-square test of independence of variables in a contingency table
- H0: 페미니즘의 긍정/부정 여부에 따라 결혼의향 여부의 차이는 없다.
- Ha: 차이가 있다

In [80]:
# row: 결혼의향 있음/없음. col: 페미니즘에 긍정/부정/무관심
r1 = [39, 161, 127] # 결혼의향 있을때 페미니즘에 긍정/부정/무관심 인원수
r2 = [93, 86, 73] # 결혼의향 없음.
ctable = np.array([r1, r2]) 

In [81]:
# 페미니즘에 대한 인식에 따라 결혼의향의 차이가 있다
res = stats.chi2_contingency(ctable)
test_hypothesis(res[1])

Reject H0 (p-value = 1.0403e-11)


In [84]:
# 결혼의향 있을때 페미니즘에 대한 인식의 차이가 있다.
res = stats.chisquare(r1)
test_hypothesis(res[1])

Reject H0 (p-value = 1.6070e-16)


In [85]:
# 결혼의향 없을때 페미니즘에 대한 인식의 차이는 없다.
res = stats.chisquare(r2)
test_hypothesis(res[1])

Accept H0 (p-value = 2.9341e-01)


# Item 3

- 20대 여성의 49.2%, ...가 페미니즘에 긍정적 모습을 보인다. 
- (그림5) 20대 여성의 30.2%, 20.5%가 페미니즘에 각각 보통, 부정적
- (그림5) 전체 응답자중 18.8%가 페미니즘에 긍정적, 41.3%가 부정적
- (그림5) 결혼의향 없는 미혼 중 37%가 페미니즘에 긍정적, 34.3%가 부정적
- 페미니즘은 결혼 의향과 상당한 상관관계를 보인다. 페미니즘에 긍정적인 응답자 가운데 70.1%가 결혼 의향이 없다고 밝혔다. 반면 페미니즘에 ‘보통’이거나 ‘부정적’인 태도를 갖는 사람들은 각각 63.5%, 65.2%가 결혼 의향이 있다고 응답했다
- 페미니즘에 긍정적인 태도를 가진 응답자 가운데 63.3%는 연애에 대해서도 ‘반드시 할 필요 없다’는 응답을 보였다.
- 흥미로운 지점은 연애나 결혼 상대방을 고르는 과정에서 페미니즘이 강한 변수로 작동한다는 것이다. 페미니즘에 ‘긍정적’이라고 응답한 사람들 중 54.5%가 페미니즘에 대한 태도를 배우자 선택의 조건으로 꼽았다. 반대로 페미니즘에 ‘부정적’인 응답을 한 사람들 가운데 58.6%가 페미니즘에 대한 태도를 배우자 선택의 조건으로 꼽는다. 페미니즘에 대해 ‘보통’ 의견을 갖고 있는 경우, 배우자 선택 기준으로 페미니즘은 그리 중요하지 않다(17.9%). 페미니즘에 대한 강한 의견(긍정 또는 부정)을 가질수록, 결혼 상대방의 태도를 민감하게 여긴다. (그림6도 참고)
- 20대 여성에게 특히 정치적 성향은 중요한 요건이다. 20대 여성 응답자 가운데 58.1%가 연애 상대방의 정치적 성향이 중요하다고 답했고, 66%는 결혼 상대방의 정치적 성향이 중요하다고 답했다. 같은 기준 20대 남성의 응답이 연애의 경우 42.2%, 결혼의 경우 40.9%라는 점을 감안하면 20대에서도 특히 여성들이 페미니즘과 같은 정치적 이슈에 대한 태도를 파트너 선택의 중요한 요건으로 삼는다는 걸 알 수 있다.

In [4]:
c_level = .95 # 신뢰수준 95%
alpha = 1 - c_level 

In [5]:
n_all = 1000
n_single = 579 # 미혼남녀
n_20s_f = 160 # 20대 여성 응답자(160명)

In [6]:
# 미혼 응답자 579명 중 결혼 의향이 있다는 사람은 56.5%, 의향이 없다는 사람은 43.5%였다. 
n_single_marry = round(n_single * .567)
n_single_nomarry = n_single - n_single_marry

In [7]:
# 그림5
n_fem = round(n_all * 0.188) # 전체중 페미니즘에 긍정적 인원수
n_fema = round(n_all * 0.413) # 전체중 페미니즘에 부정적 인원수
n_single_nomarry_fem = round(n_single_nomarry * .37) # 결혼의향 없는 미혼중 페미니즘에 긍정적
n_single_nomarry_fema = round(n_single_nomarry * .343) # 결혼의향 없는 미혼중 페미니즘에 부정적

In [8]:
# 그림5: 페미니즘 태도 비율: 긍정/보통/부정 순
r1 = [.188, .399, .413] # 전체 1000명 
r2 = [.095, .308, .597] # 20대 남성
r3 = [.492, .302, .205] # 20대 여성
r4 = [.370, .287, .343] # 결혼 의향 없는 미혼

In [9]:
def get_freq(r, n):
    r = [x/sum(r) for x in r] # check sum of ratio
    res = [round(x * n) for x in r]
    s = sum(res)
    if s != n:
        print(f'WARNING: sum of freqs {s} != {n}')
    return res

## ST01
20대 여성의 페미니즘에 대한 태도 검토

Result:
- 20대 여성의 페미니즘에 대한 태도는 전체, 20대남성, 결혼의향 업는 미혼 집단과 다르다고할 수 있다.

### CS1
Calculate a one-way chi-square test.

In [10]:
get_freq2 = lambda x: get_freq(x, n_20s_f)
obj = get_freq2(r3) # 관측값: 20대 여성
exp1 = get_freq2(r1) # 기대값: 전체 1000명의 태도 비율
exp2 = get_freq2(r2) # 기대값: 20대 남성
exp3 = get_freq2(r4) # 기대값: 결혼 의향 없는 미혼

In [11]:
# 20대 여성의 페미니즘에 대한 태도는 전체의 태도와 다르다
res = stats.chisquare(obj, exp1)
test_hypothesis(res[1])

Reject H0 (p-value = 1.4773e-22)


In [12]:
# 20대 여성의 페미니즘에 대한 태도는 20대 남성의 태도와 다르다
res = stats.chisquare(obj, exp2)
test_hypothesis(res[1])

Reject H0 (p-value = 5.2749e-69)


In [13]:
# 20대 여성의 페미니즘에 대한 태도는 결혼의향없는 미혼의 태도와 다르다
res = stats.chisquare(obj, exp3)
test_hypothesis(res[1])

Reject H0 (p-value = 3.9631e-04)


## ST02
결혼 의향과 페미니즘에 대한 태도 관계

Result:
- 미혼에 대하여 결혼 의향과 페미니즘에 대한 태도는 관련이 있다.
- 결혼의향 없는 미혼은 페미니즘에 대하여 긍정/보통/부정의 비율에 차이가 없다.
- 결혼의향 있는 미혼은 페미니즘에 대하여 긍정/보통/부정의 비율에 차이가 있다.

### CS1
Chi-square test of independence of variables in a contingency table.

In [14]:
# 그림5: 결혼 의향 없는 미혼의 페미니즘 태도, 긍정/보통/부정 순
obj1 = r4 # 비율
obj1 = get_freq(obj1, n_single_nomarry) # 빈도
obj1

[93, 72, 86]

In [15]:
# 결혼의향 있는 미혼의 페미니즘 태도, 긍정/보통/부정 순
obj2 = [0]*3

# 페미니즘에 긍정적인 응답자 가운데 70.1%가 결혼 의향이 없다고 밝혔다 => 93명
# 93/(x+93) = .701 where x = obj2[0]
obj2[0] = 93/.701 - 93

# 페미니즘에 ‘보통’이거나 ‘부정적’인 태도를 갖는 사람들은 각각 63.5%, 65.2%가 결혼 의향이 있다고 응답
# 결혼의향 없는 미혼중 페미니즘에 보통, 부정 => 각각 72, 86명
# 부정적인 경우 x/(x+86) = .652 where x = obj2[2]
obj2[1] = 72 * .635 / (1-.635)
obj2[2] = 86 * .652 / (1-.652)

s = sum(obj2)
obj2 = [round(x/s*n_single_marry) for x in obj2]
obj2

[40, 126, 162]

In [16]:
cols = [['페미니즘'], ['긍정','보통','부정']]
cols = pd.MultiIndex.from_product(cols)
index = [['결혼의향'], ['없음','있음']]
index = pd.MultiIndex.from_product(index)
df = pd.DataFrame(np.array([obj1, obj2]), columns=cols, index=index)
df

페미니즘          
          긍정   보통   부정
결혼의향 없음   93   72   86
     있음   40  126  162

In [17]:
# 미혼에 대하여 결혼 의향과 페미니즘에 대한 태도는 관련이 있다 
res = stats.chi2_contingency(df)
test_hypothesis(res[1])

Reject H0 (p-value = 1.5517e-11)


### CS2
Calculate a one-way chi-square test.

In [18]:
get_freq2 = lambda x: get_freq(x, n_single_nomarry)
obj = df.iloc[0].to_list() # 관측값: 결혼의향 없는 미혼
exp1 = get_freq2(r3) # 기대값: 20대 여성
exp2 = get_freq2(r1) # 기대값: 전체 1000명의 태도 비율
exp3 = get_freq2(r2) # 기대값: 20대 남성

In [19]:
#sum(exp1): 252
exp1 = [124, 75, 52]
sum(exp1)

251

In [20]:
res = stats.chisquare(obj, exp1)
test_hypothesis(res[1])

Reject H0 (p-value = 2.9087e-07)


In [21]:
res = stats.chisquare(obj, exp2)
test_hypothesis(res[1])

Reject H0 (p-value = 6.9956e-13)


In [22]:
res = stats.chisquare(obj, exp3)
test_hypothesis(res[1])

Reject H0 (p-value = 8.3820e-50)


In [23]:
# 결혼의향 없는 미혼의 페미니즘에 대한 긍정/보통/부정의 태도는 차이가 없다.
res = stats.chisquare(obj)
test_hypothesis(res[1])

Accept H0 (p-value = 2.5499e-01)


In [24]:
# 결혼의향 있는 미혼의 페미니즘에 대한 긍정/보통/부정의 태도는 차이가 있다.
obj2 = df.iloc[1].to_list() # 관측값: 결혼의향 있는 미혼
res = stats.chisquare(obj2)
test_hypothesis(res[1])

Reject H0 (p-value = 2.4654e-16)
